## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = ['/kaggle/input/117-googleq-a-train-bert-base-uncased-3f-lrwarmup2/model_fold_1.h5']
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/117-googleq-a-train-bert-base-uncased-3f-lrwarmup2/model_fold_1.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.953106,0.649150,0.178664,0.449247,0.592579,0.657209,0.747394,0.665287,0.693865,...,0.936705,0.907583,0.526517,0.974459,0.949627,0.839453,0.076374,0.058957,0.828584,0.928578
1,46,0.909277,0.574576,0.007950,0.748827,0.782847,0.932559,0.553895,0.486828,0.077453,...,0.720354,0.952953,0.645335,0.975511,0.981057,0.860066,0.947266,0.115163,0.074038,0.885491
2,70,0.931160,0.684297,0.048878,0.794811,0.851340,0.835957,0.651942,0.566282,0.138211,...,0.936293,0.925346,0.695983,0.975787,0.968447,0.867035,0.125692,0.090949,0.769627,0.909955
3,132,0.868679,0.357775,0.006426,0.616084,0.765328,0.891212,0.556151,0.460004,0.086673,...,0.669331,0.943520,0.679073,0.972396,0.984610,0.876771,0.904437,0.201310,0.255067,0.903419
4,200,0.949722,0.319533,0.076942,0.789904,0.674067,0.822705,0.703432,0.541448,0.091723,...,0.704727,0.868291,0.566735,0.964622,0.966717,0.797448,0.109058,0.075362,0.671565,0.923685


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.890797,0.574736,0.033130,0.701326,0.798093,0.834500,0.590230,0.476437,0.238610,...,0.793933,0.929824,0.665695,0.961854,0.970540,0.868646,0.539612,0.142631,0.500427,0.914043
std,2812.670060,0.055720,0.160865,0.068943,0.128651,0.114969,0.148986,0.066932,0.096878,0.231657,...,0.100598,0.027924,0.060943,0.014543,0.015083,0.043532,0.340000,0.072684,0.298891,0.023596
min,39.000000,0.688914,0.273012,0.002709,0.178733,0.263501,0.118171,0.451823,0.276137,0.004857,...,0.511694,0.808131,0.497509,0.892705,0.892680,0.695992,0.003229,0.005469,0.006342,0.813818
25%,2572.000000,0.853975,0.433471,0.007035,0.617269,0.744809,0.811626,0.543097,0.408357,0.067371,...,0.708160,0.913603,0.624777,0.954559,0.964217,0.844051,0.167201,0.092085,0.220890,0.900820
50%,5093.000000,0.897117,0.566032,0.010590,0.708153,0.808971,0.887752,0.580235,0.452422,0.146051,...,0.803749,0.934467,0.665008,0.964193,0.973911,0.873596,0.636080,0.143585,0.500229,0.915400
75%,7482.000000,0.934798,0.709137,0.021820,0.792959,0.875765,0.927121,0.632218,0.536216,0.343183,...,0.885877,0.949543,0.705000,0.972212,0.980814,0.900055,0.857060,0.190358,0.765212,0.930382
max,9640.000000,0.984523,0.898646,0.731238,0.973177,0.978348,0.978824,0.815617,0.787219,0.866693,...,0.968408,0.980825,0.857161,0.990856,0.991250,0.972859,0.978910,0.359585,0.993416,0.970926
